# No rag Output process

In [6]:
import json
import os, re

In [23]:
query = r'^Write me a bio of: .+\n\n'
list_new_out = []
with open("../results/bio/bio_writing_no_rag_post.jsonl", "w") as out_f:
    with open("../results/bio/bio_writing_no_rag.jsonl", "r") as f:
        for line in f.readlines():
            out = json.loads(line)
            json.dump({"entity": out['entity'],
                        "generated_text" :re.split(query,out['generated_text'])[1]}, out_f)
            out_f.write('\n')
            


In [24]:
with open("../results/bio/bio_writing_no_rag_post.jsonl", "r") as f:
        for line in f.readlines():
            out = json.loads(line)
            print(out)

{'entity': 'Ludwig van Beethoven', 'generated_text': 'Ludwig van Beethoven (1770-1827) was a German composer and pianist who is widely considered one of the most important figures in the history of classical music. Born in Bonn, Germany, Beethoven began studying music at a young age and quickly demonstrated a prodigious talent. He went on to study with some of the leading composers of his day, including Joseph Haydn and Wolfgang Amadeus Mozart.\n\nDespite his early success as a composer and performer, Beethoven struggled with hearing loss throughout his life. This did not stop him from continuing to compose, however, and he went on to create some of his most famous works, including his symphonies, piano sonatas, and string quartets. Beethoven\'s music was revolutionary for its time, and it helped to pave the way for the Romantic movement in music.\n\nBeethoven\'s personal life was also marked by controversy and tragedy. He was known to be fiercely independent and had difficulty forming

# Generating a HF dataset from the FactScore Bio entities

In [2]:
import json
from datasets import load_dataset

In [4]:
with open("./prompt_entities.txt", 'r') as f_in:
    with open("./prompt_entities.jsonl", 'w') as f_out:
        for line in f_in.readlines():
            line = line.strip()
            cur_dict = {"input": "Write me a bio of: {}".format(line)}
            json.dump(cur_dict,f_out)
            f_out.write("\n")

ds = load_dataset('json', data_files=["./prompt_entities.jsonl"]

In [7]:
from datasets import load_dataset



Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/lucas_dfki/.cache/huggingface/datasets/json/default-2ae1b026fdf5ee95/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

# Exploring The Generations
## Non-Rag Llama2-Chat

In [18]:
import json
from pprint import pprint
from IPython.display import clear_output

In [19]:
very_rare = []
very_freq = []
rarity_set = set()


with open("../results/bio/results_llama2Chat_bio_prompts_ctxs.jsonl", 'r') as f:
    for line in f.readlines():
        dct = json.loads(line)
        if "very rare" in dct['cat']:
            very_rare.append(dct)
        if "very freq" in dct['cat']:
            very_freq.append(dct)          
        rarity_set.add(dct['cat'][0])


print(rarity_set)
print("n v_rare: {}".format(len(very_rare)))
print("n v_freq: {}".format(len(very_rare)))

{'medium', 'very freq', 'very rare', 'rare', 'freq'}
n v_rare: 100
n v_freq: 100


In [20]:
# View rare outputs

for line in very_rare:
    # print(line.keys())
    # break
    clear_output()
    pprint(line['topic'])
    pprint(line['output'])
    break
#    a = input("for next")
#    if a == "x":
#        break
#    else:
#        continue


'Jessie Mae Brown Beavers'
('  Jessie Mae Brown Beavers (1924-2019) was a pioneering African American '
 'educator, administrator, and civil rights activist from Kentucky. She was '
 'born in rural Breathitt County, Kentucky, to sharecropper parents and grew '
 'up during the Great Depression. Despite the challenges of poverty and racial '
 'discrimination, Beavers was determined to pursue her education and make a '
 'difference in her community.\n'
 '\n'
 'Beavers attended segregated schools throughout her childhood and '
 'adolescence, where she excelled academically and developed a passion for '
 "teaching. She went on to earn a bachelor's degree in education from Kentucky "
 'State University (then known as Kentucky State College) in 1946, becoming '
 'one of the first African Americans to graduate from the institution.\n'
 '\n'
 'After completing her degree, Beavers began her teaching career in the '
 'segregated schools of Breathitt County. Despite facing discrimination and '
 'l

In [21]:
# View very freq outputs

for line in very_freq:
    # print(line.keys())
    # break
    clear_output()
    pprint(line['topic'])
    pprint(line['output'])
    break
#    if a == "x":
#        break
#    else:
#        continue


'John Forbes Nash Jr.'
('  John Forbes Nash Jr. (1928-2015) was a renowned American mathematician and '
 'economist who made significant contributions to the fields of game theory, '
 'differential geometry, and social choice theory. He is best known for his '
 'work on the Nash equilibrium, which is a fundamental concept in economics '
 'and political science. Nash was born on June 13, 1928, in Bluefield, West '
 'Virginia, and grew up in a middle-class family. He developed an interest in '
 'mathematics at an early age and attended the University of California, '
 'Berkeley, where he earned his undergraduate degree in 1950.\n'
 '\n'
 'After serving in the United States Army during the Korean War, Nash enrolled '
 'at Princeton University, where he earned his Ph.D. in mathematics in 1957. '
 'His dissertation, supervised by John von Neumann, was on the topic of game '
 'theory and the Nash equilibrium. In the early 1950s, Nash began working at '
 'the RAND Corporation, a think tank fo

In [22]:
# Saving a subset of 25 samples from the 'very rare' and 'very freq' topics each

with open ("../results/bio/slice_results_llama2Chat_bio_prompts_ctxs.jsonl", "w") as f_out:
    for sample in very_rare[:25]:
        json.dump(sample,f_out)
        f_out.write("\n")
    for sample in very_freq[:25]:
        json.dump(sample,f_out)
        f_out.write("\n")
 

## With RAG Llama2-Chat7B


In [1]:
import json
from pprint import pprint
from IPython.display import clear_output

In [2]:
very_rare = []
very_freq = []
rarity_set = set()


with open("../results/bio/rag_results_llama2Chat_bio_prompts_ctxs.jsonl", 'r') as f:
    for line in f.readlines():
        dct = json.loads(line)
        if "very rare" in dct['cat']:
            very_rare.append(dct)
        if "very freq" in dct['cat']:
            very_freq.append(dct)          
        rarity_set.add(dct['cat'][0])


print(rarity_set)
print("n v_rare: {}".format(len(very_rare)))
print("n v_freq: {}".format(len(very_rare)))

{'medium', 'very freq', 'very rare', 'rare', 'freq'}
n v_rare: 100
n v_freq: 100


In [4]:
# View rare outputs

for line in very_rare:
    # print(line.keys())
    # break
    clear_output()
    pprint(line['topic'])
    pprint(line['output'])
    # pprint(line['ctxs'])
    a = input("passa ou sai")
    if a == "x":
        break
    else:
        continue

'Jessie Mae Brown Beavers'
(' Jessie Mae Brown Beavers was born on December 11, 1947, in Millport, '
 'Alabama. She is an American former politician who served as a member of the '
 'Tennessee Senate for the 17th district from 2003 to 2017. Prior to her time '
 'in the state senate, Beavers was a state representative in the 99th through '
 'the 102nd General Assemblies. In 1990, she was a member of the Brantfort '
 'Lady Blues team that won the Ontario Ladies Hockey League Championships, and '
 'she maintained a high female average and high triple score in Ontario Indian '
 'Bowling Championship in 1973. Beavers is also a professional athlete who won '
 'the most valuable player award eight times from 1962 to 1980 and was awarded '
 'the Filmmakers Hall of Fame in 1976.\n'
 '\n'
 "Beavers' mother, Louise Beavers, was an American film and television actress "
 'who appeared in dozens of films and two hit television shows from the 1920s '
 'until 1960. Beavers was born in Cincinnati, Ohi

In [17]:
# View very freq outputs

for line in very_freq:
    # print(line.keys())
    # break
    clear_output()
    pprint(line['topic'])
    pprint(line['output'])
#    if a == "x":
#        break
#    else:
#        continue
#    break


'Nick Kyrgios'
(' Sure, here is a bio of Nick Kyrgios based on the given documents:\n'
 '\n'
 'Nick Kyrgios is an Australian professional tennis player who was born on '
 'April 27, 1995, in Canberra, Australia. He is currently ranked world No. 35 '
 "in men's singles by the Association of Tennis Professionals (ATP). Kyrgios "
 'is of Greek Orthodox Christian descent and always wears a gold necklace with '
 'a cross on it.\n'
 '\n'
 'Kyrgios began playing tennis at a young age and quickly showed promise as a '
 'promising basketball player. He represented the Australian Capital Territory '
 'and Australia in his early teens before deciding to focus solely on tennis '
 'when he was 14 years old. Two years later, he gained a full scholarship at '
 'the Australian Institute of Sport, where he was able to further develop his '
 'tennis skills.\n'
 '\n'
 'In 2013, Kyrgios relocated his training base from Canberra to Melbourne '
 'Park. He has since achieved significant success on the ATP to

In [5]:
# Saving a subset of 25 samples from the 'very rare' and 'very freq' topics each

with open ("../results/bio/slice_rag_results_llama2Chat_bio_prompts_ctxs.jsonl", "w") as f_out:
    for sample in very_rare[:25]:
        json.dump(sample, f_out)
        f_out.write("\n")
    for sample in very_freq[:25]:
        json.dump(sample, f_out)
        f_out.write("\n")
 

# FS response analysis
## RAG Slice

In [16]:
# loading the outputs for comparison with the Atomic Facts
with open ("../results/bio/slice_rag_results_llama2Chat_bio_prompts_ctxs.jsonl", "r") as f_out:
    for line in f_out.readlines():
        cur = json.loads(line)
        pprint(cur['output'])
        break

(' Jessie Mae Brown Beavers was born on December 11, 1947, in Millport, '
 'Alabama. She is an American former politician who served as a member of the '
 'Tennessee Senate for the 17th district from 2003 to 2017. Prior to her time '
 'in the state senate, Beavers was a state representative in the 99th through '
 'the 102nd General Assemblies. In 1990, she was a member of the Brantfort '
 'Lady Blues team that won the Ontario Ladies Hockey League Championships, and '
 'she maintained a high female average and high triple score in Ontario Indian '
 'Bowling Championship in 1973. Beavers is also a professional athlete who won '
 'the most valuable player award eight times from 1962 to 1980 and was awarded '
 'the Filmmakers Hall of Fame in 1976.\n'
 '\n'
 "Beavers' mother, Louise Beavers, was an American film and television actress "
 'who appeared in dozens of films and two hit television shows from the 1920s '
 'until 1960. Beavers was born in Cincinnati, Ohio, and was a member of Sigm

In [32]:
# Very Rare Entities
with open("../results/metrics/fss/llama2-chat-7B/very_rare_slice_rag_results_llama2Chat_bio_prompts_ctxs.jsonl", 'r') as f:
    for line in f.readlines():
        result = json.loads(line)
        print(result.keys())
        print("Score: {}".format(result['score']))
        print("Response Ratio: {}".format(result['respond_ratio']))
        print("Number of A.Facts per response: {}".format(result['num_facts_per_response']))

dict_keys(['score', 'respond_ratio', 'decisions', 'num_facts_per_response', 'init_score'])
Score: 0.5779705115814834
Response Ratio: 1.0
Number of A.Facts per response: 49.84


In [33]:
# Very Freq Entities
with open("../results/metrics/fss/llama2-chat-7B/very_freq_slice_rag_results_llama2Chat_bio_prompts_ctxs.jsonl", 'r') as f:
    for line in f.readlines():
        result = json.loads(line)
        print(result.keys())
        print("Score: {}".format(result['score']))
        print("Response Ratio: {}".format(result['respond_ratio']))
        print("Number of A.Facts per response: {}".format(result['num_facts_per_response']))

dict_keys(['score', 'respond_ratio', 'decisions', 'num_facts_per_response', 'init_score'])
Score: 0.8466173501442724
Response Ratio: 1.0
Number of A.Facts per response: 54.96


In [10]:
print(len(result['decisions']))
for decision in result['decisions']:
    print(decision)

25
[{'atom': 'Jessie Mae Brown Beavers was born on December 11, 1947.', 'is_supported': False}, {'atom': 'Jessie Mae Brown Beavers was born in Millport, Alabama.', 'is_supported': False}, {'atom': 'She is an American.', 'is_supported': True}, {'atom': 'She is a former politician.', 'is_supported': True}, {'atom': 'She served as a member of the Tennessee Senate.', 'is_supported': False}, {'atom': 'She served as a member of the Tennessee Senate for the 17th district.', 'is_supported': False}, {'atom': 'She served from 2003 to 2017.', 'is_supported': False}, {'atom': 'Beavers was in the state senate.', 'is_supported': False}, {'atom': 'Beavers was in the state senate before a certain time.', 'is_supported': False}, {'atom': 'Beavers was a state representative.', 'is_supported': False}, {'atom': 'Beavers was a state representative in the 99th General Assembly.', 'is_supported': False}, {'atom': 'Beavers was a state representative in the 100th General Assembly.', 'is_supported': False}, {'a

## Non RAG Slice

In [17]:
with open ("../results/bio/slice_results_llama2Chat_bio_prompts_ctxs.jsonl", "r") as f_out:
    for line in f_out.readlines():
        cur = json.loads(line)
        pprint(cur['output'])
        break

(' Jessie Mae Brown Beavers was born on December 11, 1947, in Millport, '
 'Alabama. She is an American former politician who served as a member of the '
 'Tennessee Senate for the 17th district from 2003 to 2017. Prior to her time '
 'in the state senate, Beavers was a state representative in the 99th through '
 'the 102nd General Assemblies. In 1990, she was a member of the Brantfort '
 'Lady Blues team that won the Ontario Ladies Hockey League Championships, and '
 'she maintained a high female average and high triple score in Ontario Indian '
 'Bowling Championship in 1973. Beavers is also a professional athlete who won '
 'the most valuable player award eight times from 1962 to 1980 and was awarded '
 'the Filmmakers Hall of Fame in 1976.\n'
 '\n'
 "Beavers' mother, Louise Beavers, was an American film and television actress "
 'who appeared in dozens of films and two hit television shows from the 1920s '
 'until 1960. Beavers was born in Cincinnati, Ohio, and was a member of Sigm

In [34]:
# Very rare Entities
with open("../results/metrics/fss/llama2-chat-7B/very_rare_slice_results_llama2Chat_bio_prompts_ctxs.jsonl", 'r') as f:
    for line in f.readlines():
        result = json.loads(line)
        print(result.keys())
        print("Score: {}".format(result['score']))
        print("Response Ratio: {}".format(result['respond_ratio']))
        print("Number of A.Facts per response: {}".format(result['num_facts_per_response']))

dict_keys(['score', 'respond_ratio', 'decisions', 'num_facts_per_response', 'init_score'])
Score: 0.4155927626327221
Response Ratio: 1.0
Number of A.Facts per response: 70.48


In [38]:
# Very freq entities
with open("../results/metrics/fss/llama2-chat-7B/very_freq_slice_results_llama2Chat_bio_prompts_ctxs.jsonl", 'r') as f:
    for line in f.readlines():
        result = json.loads(line)
        print(result.keys())
        print("Score: {}".format(result['score']))
        print("Response Ratio: {}".format(result['respond_ratio']))
        print("Number of A.Facts per response: {}".format(result['num_facts_per_response']))

dict_keys(['score', 'respond_ratio', 'decisions', 'num_facts_per_response', 'init_score'])
Score: 0.7603339089936619
Response Ratio: 1.0
Number of A.Facts per response: 107.88
